In [10]:
import json
import re
import time
from urllib.request import urlopen

import requests
import translators as ts
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

准备阶段

In [11]:
url="https://ieeexplore.ieee.org/document/9411225"
url="https://ieeexplore.ieee.org/document/7809977" #会议
url="https://ieeexplore.ieee.org/document/5165285" #TWC
url='https://ieeexplore.ieee.org/document/4343975'
url='https://ieeexplore.ieee.org/document/4347633'
#url='https://ieeexplore.ieee.org/document/8524609'  #没有IEE参考文献的文章
#url='https://ieeexplore.ieee.org/document/10018916/' #该文章只有两列关键词 且没有参考文献
url='https://ieeexplore.ieee.org/document/9755887/' #无法获取因子
url=url+'references#references' if url[-1]=='/' else url+'/references#references'
html=urlopen(url)
html_byte=html.read()

service = ChromeService(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
driver = webdriver.Chrome(options=options,service=service)
#driver = webdriver.Chrome(executable_path=".\\chromedriver.exe",options=webdriver.ChromeOptions().add_argument("--headless"))# type: ignore
driver.get(url)
wait = WebDriverWait(driver, 20) 
wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")

'''wait = WebDriverWait(driver, 20) # 设置最多等待10秒
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "stats-reference-link-viewArticle"))) # 等待id为some_id的元素出现
'''
html_driver = driver.page_source
driver.quit()

bs=BeautifulSoup(html_byte)
bs_driver=BeautifulSoup(html_driver,'lxml' )

Incompatible release of chromedriver (version 108.0.5359.71) detected in PATH: D:\Anaconda3\envs\spider\Scripts\chromedriver.exe


In [14]:
with open("../html/get detail test.html","wb") as f:
    f.write(html_byte)

In [15]:
f=open("..\\html\\get detail test2.html","w")
f.write(html_driver)
f.close()


获取RSI

In [ ]:
url='https://ieeexplore.ieee.org/document/8949737'
service = ChromeService(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
driver = webdriver.Chrome(options=options,service=service) # type: ignore
driver.get(url)
wait = WebDriverWait(driver, 20) 
wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")
html_driver = driver.page_source

cite_this_button=driver.find_element(By.XPATH, '//button[text()="Cite This"]')
cite_this_button.click()

ris_button = driver.find_element(By.XPATH, '//a[@class="document-tab-link" and @title="RIS"]')
ris_button.click()

wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")
element = wait.until(EC.presence_of_element_located((By.XPATH, '//pre[contains(concat(" ", normalize-space(@class), " "), "text ris-text")]')))
f=open('./a.ris','w')
f.write(element.text)
f.close()


In [ ]:
# 初始化WebDriver
driver = webdriver.Chrome()

# 启动browsermob-proxy
server = browsermobproxy.Server("./browsermob-proxy")
server.start()
proxy = server.create_proxy()

# 配置WebDriver使用代理
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))

# 启动WebDriver
driver = webdriver.Chrome(options=chrome_options)

# 启动代理并打开网页
proxy.new_har("example")
driver.get("https://example.com")

# 定位下载按钮
down_button = driver.find_element_by_xpath("//a[@class='stats-download-citations-button-download']")

# 点击下载按钮
down_button.click()

# 等待一段时间，确保下载请求完成（根据实际情况调整等待时间）
time.sleep(5)

# 获取Har文件
har = proxy.har

# 查找下载链接
download_link = None
for entry in har['log']['entries']:
    if entry['response']['content']['mimeType'] == 'application/octet-stream':
        download_link = entry['response']['url']
        break

# 打印下载链接（用于调试）
print("下载链接:", download_link)

获取摘要和翻译

In [ ]:
abstract = bs.find('meta', {'property': 'twitter:description'})['content']#.replace('n','') # type: ignore
print(abstract)

abstract_translation=ts.google(abstract,from_language="en",to_language='zh', host_url="https://translate.google.com/")
print(abstract_translation)

获取关键词

In [16]:
match=re.search(r'"Author Keywords\s*","kwd":\[.*?\]',html_byte.decode(),re.DOTALL)
match=re.search(r'\[.*\]',match.group(),re.DOTALL)
if match:
    list=json.loads(match.group())
print(list)


['Wireless localization', 'privacy preserving', 'multi-party computing', 'secret sharing']


获取参考文献

In [18]:
View_Article=bs_driver.find_all("a",{"class":'stats-reference-link-viewArticle'})
CrossRef=bs_driver.find_all("a",{"class":'stats-reference-link-crossRef'})
View_Article_list=[]
CrossRef_list=[]

for article in View_Article:
    if article:
        View_Article_list.append(article.get("href"))
        
View_Article_list=['https://ieeexplore.ieee.org'+link for link in View_Article_list]

for article in CrossRef:
    if article:
        CrossRef_list.append(article.get("href"))

print(View_Article_list,'\n',CrossRef_list)

['https://ieeexplore.ieee.org/document/9112749', 'https://ieeexplore.ieee.org/document/8692423', 'https://ieeexplore.ieee.org/document/9127162', 'https://ieeexplore.ieee.org/document/8417931', 'https://ieeexplore.ieee.org/document/9079655', 'https://ieeexplore.ieee.org/document/9040904', 'https://ieeexplore.ieee.org/document/8486221', 'https://ieeexplore.ieee.org/document/6848178', 'https://ieeexplore.ieee.org/document/9112749', 'https://ieeexplore.ieee.org/document/8692423', 'https://ieeexplore.ieee.org/document/9127162', 'https://ieeexplore.ieee.org/document/8417931', 'https://ieeexplore.ieee.org/document/9079655', 'https://ieeexplore.ieee.org/document/9040904', 'https://ieeexplore.ieee.org/document/8486221', 'https://ieeexplore.ieee.org/document/6848178'] 
 ['https://doi.org/10.1145/2942358.2942384', 'https://doi.org/10.1145/3055031.3055080', 'https://doi.org/10.1145/2942358.2942384', 'https://doi.org/10.1145/3055031.3055080']


获取期刊名

In [ ]:
Journal_Title=bs_driver.find('strong', text='Published in: ').find_next_sibling().get_text()
print(Journal_Title)

'''impact_factor_list={'IEEE Transactions on Wireless Communications':10}
if Journal_Title in impact_factor_list:
    impact_factor=str(impact_factor_list[Journal_Title])
else:
    impact_factor='null'

print(impact_factor)'''

获取DOI

In [ ]:
doi=bs_driver.find('strong', text='DOI: ').find_next_sibling().get_text()
Doi_link='https://doi.org/'+doi
print(Doi_link)

获取影响因子

In [ ]:
print(bs_driver.find(href='/browse/conferences/title/')) #会议 
print(bs_driver.find({'href':'/browse/conferences/title/'}))   #TODO 为什么和上面输出不一样
print(bs_driver.find(href="/browse/periodicals/title/")) #期刊和杂志

#print(bs_driver.find(text='Journals & Magazines').parent) #这是不行的,他们都有这个,无法区分


In [20]:
Journal_index=True

if bs_driver.find(href="/browse/periodicals/title/"):
    
    Journal_index=True 
    print('this is a Journals or Magazines paper')

    journal_link='https://ieeexplore.ieee.org'+bs_driver.find('strong', text='Published in: ').find_next_sibling().get('href')
    print(journal_link)

    driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")
    driver.get(journal_link)
    time.sleep(5)
    journal_html = driver.page_source
    driver.quit()

    f=open("../html/get_detail_test3.html","w")
    f.write(journal_html)
    f.close()

    bs_temp=BeautifulSoup(journal_html)
    impact_factor=bs_temp.find('a',{'class':'stats-jhp-impact-factor'}).find('span',{'class':'text-md-md-lh'}).get_text()
    
else :
    impact_factor=''

print(impact_factor)

this is a Journals or Magazines paper
https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=5962382


C:\Users\Anable\AppData\Local\Temp\ipykernel_42520\2868855867.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")


5.281


获取日期

In [21]:

if not Journal_index:
    Date=str(bs_driver.find('strong', text='Date of Conference: ').next_sibling) # type: ignore
else:
    Date=str(bs_driver.find('strong', text='Date of Publication:').next_sibling) # type: ignore

print(Date)


 12 April 2022 


获取标题

In [22]:
print(bs.find('title').find_next_sibling().get('content')) # type: ignore #这里用.next_sibling会出错,因为两个元素间有空格

To Hide Anchor’s Position in Range-Based Wireless Localization via Secret Sharing


获取文档PDF

In [23]:
doi='10.1109/TWC.2009.070566'
scihub_link='https://sci-hub.se/'+doi
pdf_html=urlopen(scihub_link).read()
bs_pdf=BeautifulSoup(pdf_html)

In [24]:
f=open('pdf.html','wb')
f.write(pdf_html)
f.close()

In [25]:
pdf_link='https:'+bs_pdf.find('embed',{'type':"application/pdf"}).get('src')
print(pdf_link)

https://zero.sci-hub.se/3452/c5f527d3fa8cc050a5e45aa2735b02ba/abdi2009.pdf#navpanes=0&view=FitH


In [26]:
import requests
import os

response = requests.get(pdf_link)

In [28]:
file_name = os.path.basename(pdf_link)+'.pdf'
with open('..\\pdf\\'+file_name, 'wb') as f:
    f.write(response.content)